In [28]:
import torch, torchvision
import torch.nn as nn
from torchvision import datasets, models, transforms
import numpy as np

def create_body(architecture):
    if architecture == 'resnet34':
      model = models.resnet34(pretrained=True)
    elif architecture == 'resnet18':
      model = models.resnet18(pretrained=True)
    "Cut off the body of a typically pretrained `model` at `cut` (int) or cut the model as specified by `cut(model)` (function)."
    return nn.Sequential(*list(model.children())[:-1])

class conv_net(nn.Module):
    def __init__(self,fc_neurons,architecture):
        super().__init__()
        self.loss = []
        self.acc = []

        self.body = create_body(architecture)

        conv_out_size = self._get_conv_out([3,224,224])
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, fc_neurons),
            nn.ReLU(),
            nn.Linear(fc_neurons, 2),
            nn.LogSoftmax()
        )

    def _get_conv_out(self, shape):
        o = self.body(torch.zeros(1, *shape))
        return int(np.prod(o.size()))

    def forward(self, x):
        conv_out = self.body(x).view(x.size()[0], -1)
        return self.fc(conv_out)


    def fit(self, train_loader, valid_loader, lr, epochs):
        criterion = nn.NLLLoss()
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
        epochs = epochs
        print_every = 50
        for e in range(epochs):
            running_loss = 0
            for step, (x,y) in enumerate(train_loader):
                y = y.to(device)
                x = x.to(device)
                
                optimizer.zero_grad()
                
                # Forward and backward passes
                output = self.forward(x)
                loss = criterion(output, y)
                loss.backward()
                optimizer.step()
                
                running_loss += loss.item()
                
            if step > 1:
                if step % print_every == 0: 
                    print("Epoch: {}/{}... ".format(e+1, epochs),
                    "Loss: {:.4f}".format(running_loss/print_every))
                    run.log('Training Loss', running_loss/print_every)
                    running_loss = 0
                
            loss, acc = self.score(valid_loader)
            self.loss.append(loss)
            self.acc.append(acc)
            
    def score(self, valid_loader):
        test_loss = 0
        correct = 0
        preds_c = []
        with torch.no_grad():
          for data, target in valid_loader:
              data = data.to(device)
              target = target.to(device)
              output = self(data)
              test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
              pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
              preds_c.append(pred)
              correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(valid_loader.dataset)

        test_acc = 100. * correct / len(valid_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
          test_loss, correct, len(valid_loader.dataset),test_acc))
        return test_loss, test_acc

transforms = transforms.transforms.Compose([transforms.Resize([224,224]),transforms.ToTensor()])

In [29]:
import pandas as pd
import os

In [30]:
submission = pd.read_csv('/Users/paulbruffett/code/ML/invasive/sample_submission.csv')

In [31]:
submission

,name,invasive
0,1,0.5
1,2,0.5
2,3,0.5
3,4,0.5
4,5,0.5
...,...,...
1526,1527,0.5
1527,1528,0.5
1528,1529,0.5
1529,1530,0.5


In [32]:
model = conv_net(256,'resnet18')

model.load_state_dict(torch.load('invasive-model.state', map_location=torch.device('cpu')))


<All keys matched successfully>

In [33]:
from PIL import Image
import torchvision.transforms.functional as TF
PATH = '/Users/paulbruffett/code/ML/invasive/test/test/300.jpg'
image = Image.open(PATH)
x = TF.resize(image, [224,224])
x = TF.to_tensor(image)
x.unsqueeze_(0)
print(x.shape)

output = model(x)

torch.Size([1, 3, 866, 1154])


/Users/paulbruffett/anaconda3/envs/universe/lib/python3.6/site-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [34]:
output.argmax(dim=1, keepdim=True)

tensor([[0]])

In [35]:
from PIL import Image
model.eval()
PATH = '/Users/paulbruffett/code/ML/invasive/test/test/'

        
def predict_class(name):
    image = PATH+str(name)+'.jpg'
    print(image)
    image = Image.open(image).convert('RGB')
    x = transforms(image)
    x.unsqueeze_(0)
    pred = model(x)
    pred = pred.argmax(dim=1, keepdim=True)
    return pred.tolist()[0][0]

In [36]:
submission['invasive'] = submission['name'].map(predict_class)

/Users/paulbruffett/code/ML/invasive/test/test/1.jpg
/Users/paulbruffett/code/ML/invasive/test/test/2.jpg
/Users/paulbruffett/code/ML/invasive/test/test/3.jpg
/Users/paulbruffett/code/ML/invasive/test/test/4.jpg
/Users/paulbruffett/code/ML/invasive/test/test/5.jpg
/Users/paulbruffett/code/ML/invasive/test/test/6.jpg
/Users/paulbruffett/code/ML/invasive/test/test/7.jpg
/Users/paulbruffett/code/ML/invasive/test/test/8.jpg
/Users/paulbruffett/code/ML/invasive/test/test/9.jpg
/Users/paulbruffett/code/ML/invasive/test/test/10.jpg
/Users/paulbruffett/code/ML/invasive/test/test/11.jpg
/Users/paulbruffett/code/ML/invasive/test/test/12.jpg
/Users/paulbruffett/code/ML/invasive/test/test/13.jpg
/Users/paulbruffett/code/ML/invasive/test/test/14.jpg
/Users/paulbruffett/code/ML/invasive/test/test/15.jpg
/Users/paulbruffett/code/ML/invasive/test/test/16.jpg
/Users/paulbruffett/code/ML/invasive/test/test/17.jpg
/Users/paulbruffett/code/ML/invasive/test/test/18.jpg
/Users/paulbruffett/code/ML/invasive/

/Users/paulbruffett/code/ML/invasive/test/test/153.jpg
/Users/paulbruffett/code/ML/invasive/test/test/154.jpg
/Users/paulbruffett/code/ML/invasive/test/test/155.jpg
/Users/paulbruffett/code/ML/invasive/test/test/156.jpg
/Users/paulbruffett/code/ML/invasive/test/test/157.jpg
/Users/paulbruffett/code/ML/invasive/test/test/158.jpg
/Users/paulbruffett/code/ML/invasive/test/test/159.jpg
/Users/paulbruffett/code/ML/invasive/test/test/160.jpg
/Users/paulbruffett/code/ML/invasive/test/test/161.jpg
/Users/paulbruffett/code/ML/invasive/test/test/162.jpg
/Users/paulbruffett/code/ML/invasive/test/test/163.jpg
/Users/paulbruffett/code/ML/invasive/test/test/164.jpg
/Users/paulbruffett/code/ML/invasive/test/test/165.jpg
/Users/paulbruffett/code/ML/invasive/test/test/166.jpg
/Users/paulbruffett/code/ML/invasive/test/test/167.jpg
/Users/paulbruffett/code/ML/invasive/test/test/168.jpg
/Users/paulbruffett/code/ML/invasive/test/test/169.jpg
/Users/paulbruffett/code/ML/invasive/test/test/170.jpg
/Users/pau

/Users/paulbruffett/code/ML/invasive/test/test/302.jpg
/Users/paulbruffett/code/ML/invasive/test/test/303.jpg
/Users/paulbruffett/code/ML/invasive/test/test/304.jpg
/Users/paulbruffett/code/ML/invasive/test/test/305.jpg
/Users/paulbruffett/code/ML/invasive/test/test/306.jpg
/Users/paulbruffett/code/ML/invasive/test/test/307.jpg
/Users/paulbruffett/code/ML/invasive/test/test/308.jpg
/Users/paulbruffett/code/ML/invasive/test/test/309.jpg
/Users/paulbruffett/code/ML/invasive/test/test/310.jpg
/Users/paulbruffett/code/ML/invasive/test/test/311.jpg
/Users/paulbruffett/code/ML/invasive/test/test/312.jpg
/Users/paulbruffett/code/ML/invasive/test/test/313.jpg
/Users/paulbruffett/code/ML/invasive/test/test/314.jpg
/Users/paulbruffett/code/ML/invasive/test/test/315.jpg
/Users/paulbruffett/code/ML/invasive/test/test/316.jpg
/Users/paulbruffett/code/ML/invasive/test/test/317.jpg
/Users/paulbruffett/code/ML/invasive/test/test/318.jpg
/Users/paulbruffett/code/ML/invasive/test/test/319.jpg
/Users/pau

/Users/paulbruffett/code/ML/invasive/test/test/453.jpg
/Users/paulbruffett/code/ML/invasive/test/test/454.jpg
/Users/paulbruffett/code/ML/invasive/test/test/455.jpg
/Users/paulbruffett/code/ML/invasive/test/test/456.jpg
/Users/paulbruffett/code/ML/invasive/test/test/457.jpg
/Users/paulbruffett/code/ML/invasive/test/test/458.jpg
/Users/paulbruffett/code/ML/invasive/test/test/459.jpg
/Users/paulbruffett/code/ML/invasive/test/test/460.jpg
/Users/paulbruffett/code/ML/invasive/test/test/461.jpg
/Users/paulbruffett/code/ML/invasive/test/test/462.jpg
/Users/paulbruffett/code/ML/invasive/test/test/463.jpg
/Users/paulbruffett/code/ML/invasive/test/test/464.jpg
/Users/paulbruffett/code/ML/invasive/test/test/465.jpg
/Users/paulbruffett/code/ML/invasive/test/test/466.jpg
/Users/paulbruffett/code/ML/invasive/test/test/467.jpg
/Users/paulbruffett/code/ML/invasive/test/test/468.jpg
/Users/paulbruffett/code/ML/invasive/test/test/469.jpg
/Users/paulbruffett/code/ML/invasive/test/test/470.jpg
/Users/pau

/Users/paulbruffett/code/ML/invasive/test/test/603.jpg
/Users/paulbruffett/code/ML/invasive/test/test/604.jpg
/Users/paulbruffett/code/ML/invasive/test/test/605.jpg
/Users/paulbruffett/code/ML/invasive/test/test/606.jpg
/Users/paulbruffett/code/ML/invasive/test/test/607.jpg
/Users/paulbruffett/code/ML/invasive/test/test/608.jpg
/Users/paulbruffett/code/ML/invasive/test/test/609.jpg
/Users/paulbruffett/code/ML/invasive/test/test/610.jpg
/Users/paulbruffett/code/ML/invasive/test/test/611.jpg
/Users/paulbruffett/code/ML/invasive/test/test/612.jpg
/Users/paulbruffett/code/ML/invasive/test/test/613.jpg
/Users/paulbruffett/code/ML/invasive/test/test/614.jpg
/Users/paulbruffett/code/ML/invasive/test/test/615.jpg
/Users/paulbruffett/code/ML/invasive/test/test/616.jpg
/Users/paulbruffett/code/ML/invasive/test/test/617.jpg
/Users/paulbruffett/code/ML/invasive/test/test/618.jpg
/Users/paulbruffett/code/ML/invasive/test/test/619.jpg
/Users/paulbruffett/code/ML/invasive/test/test/620.jpg
/Users/pau

...

/Users/paulbruffett/code/ML/invasive/test/test/904.jpg
/Users/paulbruffett/code/ML/invasive/test/test/905.jpg
/Users/paulbruffett/code/ML/invasive/test/test/906.jpg
/Users/paulbruffett/code/ML/invasive/test/test/907.jpg
/Users/paulbruffett/code/ML/invasive/test/test/908.jpg
/Users/paulbruffett/code/ML/invasive/test/test/909.jpg
/Users/paulbruffett/code/ML/invasive/test/test/910.jpg
/Users/paulbruffett/code/ML/invasive/test/test/911.jpg
/Users/paulbruffett/code/ML/invasive/test/test/912.jpg
/Users/paulbruffett/code/ML/invasive/test/test/913.jpg
/Users/paulbruffett/code/ML/invasive/test/test/914.jpg
/Users/paulbruffett/code/ML/invasive/test/test/915.jpg
/Users/paulbruffett/code/ML/invasive/test/test/916.jpg
/Users/paulbruffett/code/ML/invasive/test/test/917.jpg
/Users/paulbruffett/code/ML/invasive/test/test/918.jpg
/Users/paulbruffett/code/ML/invasive/test/test/919.jpg
/Users/paulbruffett/code/ML/invasive/test/test/920.jpg
/Users/paulbruffett/code/ML/invasive/test/test/921.jpg
/Users/pau

/Users/paulbruffett/code/ML/invasive/test/test/1054.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1055.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1056.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1057.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1058.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1059.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1060.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1061.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1062.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1063.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1064.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1065.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1066.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1067.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1068.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1069.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1070.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1

/Users/paulbruffett/code/ML/invasive/test/test/1203.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1204.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1205.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1206.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1207.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1208.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1209.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1210.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1211.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1212.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1213.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1214.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1215.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1216.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1217.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1218.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1219.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1

/Users/paulbruffett/code/ML/invasive/test/test/1352.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1353.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1354.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1355.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1356.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1357.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1358.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1359.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1360.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1361.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1362.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1363.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1364.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1365.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1366.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1367.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1368.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1

/Users/paulbruffett/code/ML/invasive/test/test/1499.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1500.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1501.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1502.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1503.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1504.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1505.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1506.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1507.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1508.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1509.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1510.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1511.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1512.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1513.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1514.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1515.jpg
/Users/paulbruffett/code/ML/invasive/test/test/1

In [37]:
submission['invasive'].unique()

array([0, 1])

In [38]:
def reverse(x):
    if x==1:
        return 0
    else:
        return 1
    
submission['invasive'] = submission['invasive'].apply(reverse)

In [39]:
submission.to_csv('predictions.csv')